In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.2.0
sys.version_info(major=3, minor=8, micro=2, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.5
pandas 1.0.4
sklearn 0.23.1
tensorflow 2.2.0
tensorflow.keras 2.3.0-tf


In [7]:
# 定义一个对数据规范化的线性运算单元函数，对负数进行规范化
def scaled_elu(z, scale=1.0, alpha=1.0):
    # 使用tf.greater对z进行判断，是否为正数
    is_positive = tf.greater_equal(z, 0.0)
    
    # z >= 0 ? scale * z : scale * alpha * tf.nn.elu(z)
    # tf.nn.elu(z)=exp(z)-1, if z < 0, otherwise z
    return scale * tf.where(is_positive, z, alpha * tf.nn.elu(z))

print(scaled_elu(tf.constant(3.)))
print(scaled_elu(tf.constant([-2., 6])))

# 使用tf.function将指定函数转换为tf的图函数
scaled_elu_tf = tf.function(scaled_elu)
print(scaled_elu_tf(tf.constant(3.)))
print(scaled_elu_tf(tf.constant([-2., 6])))  # 求解结果一致，但是性能有一定的提升

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor([-0.86466473  6.        ], shape=(2,), dtype=float32)
tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor([-0.86466473  6.        ], shape=(2,), dtype=float32)


In [12]:
# 在转换后的函数后加上.python_function可以使用原来python的函数
print(scaled_elu_tf.python_function is scaled_elu)
print(scaled_elu_tf.python_function(tf.constant([-2., 6]))) 

True
tf.Tensor([-0.86466473  6.        ], shape=(2,), dtype=float32)


In [8]:
%timeit scaled_elu(tf.random.normal((1000, 1000)))
%timeit scaled_elu_tf(tf.random.normal((1000, 1000)))

14.4 ms ± 580 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
12.2 ms ± 194 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
# 使用@tf.function装饰器就会将函数转换为图函数进行运算
@tf.function
def converge_to_2(n_iters):
    total = tf.constant(0.)
    increment = tf.constant(1.)
    for _ in range(n_iters):
        total += increment
        increment /= 2.0
    return total

print(converge_to_2(30))

tf.Tensor(2.0, shape=(), dtype=float32)


In [16]:
# 显示函数转换后的代码
def display_tf_code(func):
    code = tf.autograph.to_code(func)
    from IPython.display import display, Markdown
    display(Markdown('```python\n{}\n```'.format(code)))


In [18]:
display_tf_code(scaled_elu)

```python
def tf__scaled_elu(z, scale=None, alpha=None):
    do_return = False
    retval_ = ag__.UndefinedReturnValue()
    with ag__.FunctionScope('scaled_elu', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        is_positive = ag__.converted_call(tf.greater_equal, (z, 0.0), None, fscope)
        try:
            do_return = True
            retval_ = fscope.mark_return_value((scale * ag__.converted_call(tf.where, (is_positive, z, (alpha * ag__.converted_call(tf.nn.elu, (z,), None, fscope))), None, fscope)))
        except:
            do_return = False
            raise
    (do_return,)
    return ag__.retval(retval_)

```

In [20]:
display_tf_code(converge_to_2.python_function)  # 如果已经转换了的话，会报错

```python
def tf__converge_to_2(n_iters):
    do_return = False
    retval_ = ag__.UndefinedReturnValue()
    with ag__.FunctionScope('converge_to_2', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        total = ag__.converted_call(tf.constant, (0.0,), None, fscope)
        increment = ag__.converted_call(tf.constant, (1.0,), None, fscope)

        def get_state():
            return (total, increment)

        def set_state(loop_vars):
            nonlocal total, increment
            (total, increment) = loop_vars

        def loop_body(itr):
            nonlocal total, increment
            _ = itr
            total += increment
            increment /= 2.0
        ag__.for_stmt(ag__.converted_call(range, (n_iters,), None, fscope), None, loop_body, get_state, set_state, ('total', 'increment'), {})
        try:
            do_return = True
            retval_ = fscope.mark_return_value(total)
        except:
            do_return = False
            raise
    (do_return,)
    return ag__.retval(retval_)

```

In [21]:
# 将变量定义在函数外的原因
# 1.避免因为多次调用函数而使用过多的变量空间，占用内存
# 2.避免因为多次调用函数，造成运算冲突

var = tf.Variable(0.)

@tf.function
def add_66():
    return var.assign_add(66)

print(add_66())

tf.Tensor(66.0, shape=(), dtype=float32)


In [36]:
# 在tf.function装饰器中加入input_signature可以限制数据类型

@tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name='x')])
def cube(z):
    return tf.pow(z, 3)

# 浮点数
try:
    print(cube(tf.constant([1., 2., 3.])))
except ValueError as ve:
    print(ve)
    
# 整数
# print(cube(tf.constant([1, 2, 3])))

Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name='x'))


In [41]:
# 使用.get_concrete_function()将数据类型以参数方式限制内嵌到函数中
cube_func_int32 = cube.get_concrete_function(
    tf.TensorSpec([None], tf.int32))

print(cube_func_int32)
print(type(cube_func_int32))

<class 'tensorflow.python.eager.function.ConcreteFunction'>


In [43]:
print(cube_func_int32 is cube.get_concrete_function(
    tf.TensorSpec([5], tf.int32)))
print(cube_func_int32 is cube.get_concrete_function(
    tf.constant([1, 2, 3])))

True
True


In [44]:
cube_func_int32.graph

In [45]:
# 图函数下的操作
cube_func_int32.graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Pow/y' type=Const>,
 <tf.Operation 'Pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [53]:
pow_op = cube_func_int32.graph.get_operations()[2]
print(pow_op)

name: "Pow"
op: "Pow"
input: "x"
input: "Pow/y"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}



In [54]:
x_op = cube_func_int32.graph.get_operations()[0]
print(x_op)

name: "x"
op: "Placeholder"
attr {
  key: "_user_specified_name"
  value {
    s: "x"
  }
}
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
    }
  }
}



In [56]:
powy_op = cube_func_int32.graph.get_operations()[1]
print(powy_op)

name: "Pow/y"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_INT32
      tensor_shape {
      }
      int_val: 3
    }
  }
}



In [57]:
print(list(pow_op.inputs))
print(list(pow_op.outputs))

[<tf.Tensor 'x:0' shape=(None,) dtype=int32>, <tf.Tensor 'Pow/y:0' shape=() dtype=int32>]
[<tf.Tensor 'Pow:0' shape=(None,) dtype=int32>]
